In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp dlmm

# DLMM

> A Dynamic Linear Mixture Model, or DLMM, is the combination of a Bernoulli DGLM and a Normal DLM as described in [Yanchenko, Deng, Li, Cron and West (2021)](https://arxiv.org/pdf/2101.03408.pdf).

The DLMM is a combination of a Bernoulli DGLM and a Normal DLM and is motivated by and similar to the DCMM [Berry and West (2019)](https://arxiv.org/pdf/1805.05232.pdf). The Bernoulli DGLM models the probability of the observation being zero. Conditional on a non-zero outcome, then the continuous observation follows a Normal distribution. This is useful for modeling continuous-valued time series with some observations that are exactly zero; under a Normal distribution, the probability of any one observation being identically zero is 0. For example, DLMMs can be used to model series of log total spend, where at some time points, the total spend is identically $0. 

Formally, a DLMM models observations $y_t$ as:
$$
\quad z_{t} \sim Bern(\pi_{t}) \quad \textrm{and}\quad y_{t} | z_{t}  = 
\begin{cases}
0, & \text{if } z_{t} = 0,\\
x_{t}, \quad x_{t} \sim \mathcal{N}(\mathbf{F}_t'\boldsymbol{\theta}_t, \mathbf{V}_t), & \textrm{if}\ z_{t} = 1.
\end{cases}
$$

$\boldsymbol{\theta}_t$ follows a DLM evolution, involving known regression vectors  $\mathbf{F}_t$.

Latent factors are not currently supported for the DLMM.

In [ ]:
#hide
#exporti
import numpy as np
from pybats.dglm import bern_dglm, dlm
from pybats.update import update_F
from scipy.special import expit

In [ ]:
#export
class dlmm:
    def __init__(self,
                 a0_bern = None,
                 R0_bern = None,
                 nregn_bern = 0,
                 ntrend_bern = 0,
                 nhol_bern = 0,
                 seasPeriods_bern = [],
                 seasHarmComponents_bern = [],
                 deltrend_bern = 1, delregn_bern = 1,
                 delhol_bern = 1,
                 delseas_bern = 1, 
                 rho = 1,
                 a0_dlm = None,
                 R0_dlm = None,
                 nregn_dlm = 0,
                 ntrend_dlm = 0,
                 nhol_dlm = 0,
                 seasPeriods_dlm = [],
                 seasHarmComponents_dlm = [],
                 deltrend_dlm = 1, delregn_dlm = 1,
                 delhol_dlm = 1,
                 delseas_dlm = 1,
                 delVar_dlm = 1,
                 interpolate=True,
                 adapt_discount=False):
        """
        :param a0_bern: Prior mean vector for bernoulli DGLM
        :param R0_bern: Prior covariance matrix for bernoulli DGLM
        :param nregn_bern: Number of regression components in bernoulli DGLM
        :param ntrend_bern: Number of trend components in bernoulli DGLM
        :param seasPeriods_bern: List of periods of seasonal components in bernoulli DGLM
        :param seasHarmComponents_bern: List of harmonic components included for each period in bernoulli DGLM
        :param deltrend_bern: Discount factor on trend components in bernoulli DGLM
        :param delregn_bern: Discount factor on regression components in bernoulli DGLM
        :param delhol_bern: Discount factor on holiday component in bernoulli DGLM (currently deprecated)
        :param delseas_bern: Discount factor on seasonal components in bernoulli DGLM
        :param rho: random effect discount factor for bernoulli DGLM (smaller rho increases variance)
        :param a0_dlm: Prior mean vector for normal DLM
        :param R0_dlm: Prior covariance matrix for normal DLM
        :param nregn_dlm: Number of regression components in normal DLM
        :param ntrend_dlm: Number of trend components in normal DLM
        :param seasPeriods_dlm: List of periods of seasonal components in normal DLM
        :param seasHarmComponents_dlm: List of harmonic components included for each period in normal DLM
        :param deltrend_dlm: Discount factor on trend components in normal DLM
        :param delregn_dlm: Discount factor on regression components in normal DLM
        :param delhol_dlm: Discount factor on holiday component in normal DLM (currently deprecated)
        :param delseas_dlm: Discount factor on seasonal components in normal DLM
        :param delVar_dlm: Discount factor for observation volatility in normal DLM
        """

        self.bern_mod = bern_dglm(a0=a0_bern,
                                  R0=R0_bern,
                                  nregn=nregn_bern,
                                  ntrend=ntrend_bern,
                                  nlf=0,
                                  nhol=nhol_bern,
                                  seasPeriods=seasPeriods_bern,
                                  seasHarmComponents=seasHarmComponents_bern,
                                  deltrend=deltrend_bern, delregn=delregn_bern,
                                  delhol=delhol_bern, delseas=delseas_bern,
                                  dellf=1,
                                  rho = rho,
                                  interpolate=interpolate,
                                  adapt_discount=adapt_discount)

        self.dlm_mod = dlm(a0=a0_dlm,
                           R0=R0_dlm,
                           nregn=nregn_dlm,
                           ntrend=ntrend_dlm,
                           nhol=nhol_dlm,
                           seasPeriods=seasPeriods_dlm,
                           seasHarmComponents=seasHarmComponents_dlm,
                           deltrend=deltrend_dlm, delregn=delregn_dlm,
                           delhol=delhol_dlm, delseas=delseas_dlm,
                           dellf=1,
                           delVar = delVar_dlm,
                           interpolate=interpolate,
                           adapt_discount=adapt_discount)

        self.t = 0

        
    # X is a list or tuple of length 2. The first component is data for the bernoulli DGLM, the next is for the Normal DLM.
    def update(self, y = None, X = None):
        X = self.make_pair(X)
        if y is None:
            self.bern_mod.update(y=y)
            self.dlm_mod.update(y=y)
        elif y == 0:
            self.bern_mod.update(y = 0, X = X[0])
            self.dlm_mod.update(y = np.nan, X = X[1])
        else: # only update beta model if we have significant uncertainty in the forecast
            # get the lower end forecast on the logit scale
            F = update_F(self.bern_mod, X[0], F=self.bern_mod.F.copy())
            ft, qt = self.bern_mod.get_mean_and_var(F, self.bern_mod.a, self.bern_mod.R)
            fcast_logit_lb = ft - np.sqrt(qt)
            # translate to a prod for a rough idea of whether we're already pretty confident for this forecast
            if expit(fcast_logit_lb) < 0.975:
                self.bern_mod.update(y=1, X = X[0])
            else:
                self.bern_mod.update(y=np.nan, X=X[0])
            self.dlm_mod.update(y = y, X = X[1]) # NO-Shifted Y values in the Normal DLM
        self.t += 1
        
      
    def forecast_marginal(self, k, X = None, nsamps = 1, mean_only = False, state_mean_var = False):
        X = self.make_pair(X)

        if mean_only:
            mean_bern = self.bern_mod.forecast_marginal(k, X[0], nsamps, mean_only)
            mean_dlm = self.dlm_mod.forecast_marginal(k, X[1], nsamps, mean_only)
            return mean_bern * (mean_dlm)
        elif state_mean_var:
            mv_bern = self.bern_mod.forecast_marginal(k, X[0], state_mean_var = state_mean_var)
            mv_dlm = self.dlm_mod.forecast_marginal(k, X[1], state_mean_var = state_mean_var)
            return mv_bern, mv_dlm
        else:
            samps_bern = self.bern_mod.forecast_marginal(k, X[0], nsamps)
            samps_dlm = self.dlm_mod.forecast_marginal(k, X[1], nsamps) # NO Shifted Y values in the normal DLM
            return samps_bern * samps_dlm
    
    
    def forecast_path(self, k, X = None, nsamps = 1):
        X = self.make_pair(X)

        samps_bern = self.bern_mod.forecast_path(k, X[0], nsamps)
        samps_dlm = self.dlm_mod.forecast_path(k, X[1], nsamps) # NO Shifted Y values in the Normal DLM
        return samps_bern * samps_dlm
    
    def forecast_path_copula(self, k, X = None, nsamps = 1, **kwargs):
        X = self.make_pair(X)

        samps_bern = self.bern_mod.forecast_path_copula(k, X[0], nsamps, **kwargs)
        samps_dlm = self.dlm_mod.forecast_path_copula(k, X[1], nsamps, **kwargs) # NO Shifted Y values in the Normal DLM
        return samps_bern * samps_dlm
    

    def forecast_state_mean_and_var(self, k = 1, X = None):
        mean_var_bern = self.bern_mod.forecast_state_mean_and_var(k, X[0])
        mean_var_dlm = self.dlm_mod.forecast_state_mean_and_var(k, X[1])
        return mean_var_bern, mean_var_dlm

    def make_pair(self, x):
        if isinstance(x, (list, tuple)):
            if len(x) == 2:
                return x
            else:
                return (x, x)
        else:
            return (x, x)

A DLMM can be used in the same way as a DGLM, with the standard methods `dlmm.update`, `dlmm.forecast_marginal`, and `dlmm.forecast_path`. There are equivalent helper functions as well. A full analysis can be run with `analysis_dlmm`, and `define_dlmm` helps to initialize a DLMM. These helper functions assume that the same predictors `X` are used for the Bernoulli DGLM and the Normal DLM.

The only difference from using a standard `dglm` is that outside of `analysis_dlmm`, the update and forecast functions do not automatically recognize whether the DLMM calls a copula for path forecasting. This means that the modeler needs to be more explicit in calling the correct method, such as `dlmm.forecast_path_copula` for path forecasting with a copula. The DLMM does not currently support latent factors.

A quick example of using `analysis_dlmm` to model simulated sales data follows. 

In [ ]:
import pandas as pd
import numpy as np

from pybats.shared import load_sales_example2
from pybats.analysis import analysis_dlmm

data = load_sales_example2()
data.head()

,Sales,Price,Promotion
Date,,,
2014-06-01,15.0,1.11,0.0
2014-06-02,13.0,2.19,0.0
2014-06-03,6.0,0.23,0.0
2014-06-04,2.0,-0.05,1.0
2014-06-05,6.0,-0.14,0.0


In [ ]:
# ## Convert sales to a simulated log total spend
data['Sales'] = np.where(data['Sales'] > 0, np.log(data['Sales']*np.random.uniform(0,1,1)), 0)

/Users/isaaclavine/opt/anaconda3/envs/pybats/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
prior_length = 25   # Number of days of data used to set prior
k = 7               # Forecast horizon
rho = 0.5           # Random effect discount factor to increase variance of forecast distribution
forecast_samps = 1000  # Number of forecast samples to draw
forecast_start = pd.to_datetime('2018-01-01') # Date to start forecasting
forecast_end = pd.to_datetime('2018-05-01')   # Date to stop forecasting

In [ ]:
#mod, samples = analysis_dlmm(data['Sales'].values, data[['Price', 'Promotion']].values,
#                              prior_length, k, forecast_start, forecast_end,
#                              nsamps=forecast_samps,
#                              seasPeriods=[7], seasHarmComponents=[[1,2,3]],
#                              dates=data.index,
#                              rho=rho,
#                              ret = ['model', 'forecast'])

Because the DLMM is effectively a container for a Bernoulli DGLM and a Normal DLM, we can access each of them individually. The coefficients in the Bernoulli DGLM affect the probability of a non-zero observation, and the coefficients in the Normal DLM impact the value of any non-zero observations. 

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_dglm.ipynb.
Converted 01_update.ipynb.
Converted 02_forecast.ipynb.
Converted 03_define_models.ipynb.
Converted 04_seasonal.ipynb.
Converted 05_analysis.ipynb.
Converted 06_conjugates.ipynb.
Converted 07_point_forecast.ipynb.
Converted 08_loss_functions.ipynb.
Converted 09_plot.ipynb.
Converted 10_shared.ipynb.
Converted 11_dcmm.ipynb.
Converted 12_dbcm.ipynb.
Converted 13_latent_factor.ipynb.
Converted 14_latent_factor_fxns.ipynb.
Converted 15_dlmm.ipynb.
Converted index.ipynb.
